In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

import os

import tensorflow as tf

from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,  LSTM


In [ ]:
import yfinance as yf

In [ ]:
period = '1000d'

tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOG', 'GOOGL', 'TSLA', 'NVDA', 'PYPL', 'ADBE',
           'BTC-USD', 'ETH-USD', 'XRP-USD', 'LTC-USD', 'BCH-USD', 'BNB-USD', 'LINK-USD', 'ADA-USD', 'XLM-USD', 'SOL-USD', 'TRX-USD']

In [ ]:
def indir_ve_df_olustur(tickers, period):
    # Her bir hisse senedi için boş bir sözlük oluşturun
    pariteler = {}

    # Her bir hisse senedi için döngü oluşturun ve verileri indirin
    for ticker in tickers:
        try:
            # Hisse senedi verilerini indirin
            veri = yf.download(ticker, period=period)
            
            # Veriyi sözlüğe ekleyin
            pariteler[ticker] = veri
        except Exception as e:
            print(f"{ticker} için veri indirilirken bir hata oluştu: {str(e)}")

    # Her bir hisse senedi için ayrı bir veri çerçevesi oluşturun
    df_listesi = [veri for veri in pariteler.values()]

    return df_listesi

In [ ]:
#pariteler adında klasör oluştur :

import os 

if not os.path.exists('pariteler'):
    os.makedirs('pariteler')



In [ ]:
datasets = indir_ve_df_olustur(tickers, period)

#pariteler içine kaydet 

for i in range(len(datasets)):
    datasets[i].to_csv(f'pariteler/{tickers[i]}.csv')
    #print(f'{tickers[i]} verisi csv olarak kaydedildi')


In [ ]:
window = 60

def veri_hazirla(df, window):
    # Veri çerçevesinin kopyasını oluşturun
    df = df.copy()
    
    # Veri çerçevesine yeni sütunlar ekleyin
    for i in range(1, window + 1):
        df[f'Önceki_{i}_Kapanış'] = df['Close'].shift(i)
        df[f'Önceki_{i}_Hacim'] = df['Volume'].shift(i)
        df[f'Önceki_{i}_Açılış'] = df['Open'].shift(i)
        df[f'Önceki_{i}_Yüksek'] = df['High'].shift(i)
        df[f'Önceki_{i}_Düşük'] = df['Low'].shift(i)
        
    # NaN değerleri bırakın
    df.dropna(inplace=True)
    
    return df

# Her bir veri çerçevesi için döngü oluşturun ve verileri hazırlayın
data_windowed = [veri_hazirla(df, window) for df in datasets]


if not os.path.exists('windowed'):
    os.makedirs('windowed')

# Her bir veri çerçevesi için parite ismiyle birlikte windowed klasörüne kaydedin 
for i in range(len(data_windowed)):
    data_windowed[i].to_csv(f'windowed/{tickers[i]}_windowed.csv')
    print(f'{tickers[i]} verisi windowed olarak kaydedildi')



In [ ]:
#data_windowed

In [ ]:
#her bir veri çerçevesi için X  = features y = labels olan veriyapısını kuruyoruz (etketler  = open close hig low volume)

def veri_yapisi_olustur(df, window):
    # Veri çerçevesinin kopyasını oluşturun
    df = df.copy()
    
    # Özellikler ve etiketler oluşturun
    X = df.drop(['Open', 'Close', 'High', 'Low', 'Volume'], axis=1)
    y = df[['Open', 'Close', 'High', 'Low', 'Volume']]
    
    # Veriyi numpy dizilerine dönüştürün
    X = X.values
    y = y.values
    
    # Veriyi yeniden şekillendirin
    X = X.reshape(-1, window, X.shape[1])
    y = y.reshape(-1, 5)
    
    return X, y

# Her bir veri çerçevesi için döngü oluşturun ve veri yapılarını oluşturun
data = [veri_yapisi_olustur(df, window) for df in data_windowed]

